In [ ]:
# PROBLEM STATEMENT
#BigMart Sales Data provides a detailed view of retail operations, including item specifics, outlet details, and sales labels. This dataset empowers businesses to
#strategically tailor product modifications and marketing efforts by understanding customer preferences and optimizing resources based on outlet types and sales trends.
#With insights into diverse factors such as outlet size, location, and establishment year, businesses can make informed decisions to enhance customer engagement and maximize the impact of their retail strategies.

In [ ]:
# Filtering the warnings

import warnings
warnings.filterwarnings('ignore')

In [32]:
# Importing all the necessary libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Importing all the necessary libraries

# Library for modelling
import statsmodels.api as sm

# Library for train test split
from sklearn.model_selection import train_test_split

# Library for Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE

# Library for scaling
from sklearn.preprocessing import MinMaxScaler

# Library for Variance Inflation Factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
#Reading the data set 
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [7]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [8]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [9]:
#Adding a ‘source’ column specifying where each observation belongs
train['source']='train'
test['source']='test'

#combining the dataframes into one dataset
data = pd.concat([train, test],ignore_index=True)

print(train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


In [10]:
#Checking the first 5 rows of the data set
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [11]:
#Determining the shape of the datset
data.shape

(14204, 13)

In [12]:
#Inspecting the dataset

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

In [13]:
#Checking for count of missing values in each column
(data.isnull().sum())

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [14]:
#Lets look at some basic statistics for numerical variables

data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [15]:
#Lets look at number of unique values in categorical columns

data.nunique()

Item_Identifier               1559
Item_Weight                    415
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      3
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3493
source                           2
dtype: int64

In [ ]:
#Data purify and EDA

In [16]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

In [ ]:
'Item_Weight' column:


In [17]:
# checking the number of missing values
data.Item_Weight.isnull().sum()

2439

In [18]:
data["Item_Weight"].fillna(data.groupby(by ='Item_Identifier')['Item_Weight'].transform('mean'), inplace=True)

#Checking the number of missing values
data.Item_Weight.isnull().sum()

0

In [ ]:
'Item_Fat_Content' column:

In [19]:
# checking the count of values
data.Item_Fat_Content.value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [20]:
#replacing 'LF' and 'low fat' with 'Low Fat'
#replacing 'reg' with 'Regular'
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

# checking if replacement has taken place or not
data.Item_Fat_Content.value_counts()


Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [ ]:
'Item_Visibility' column:

In [ ]:
#Replacing 0 with 'NaN'
data['Item_Visibility'] = data['Item_Visibility'].replace(0,np.NaN)

In [ ]:
#imputing the mean visibility of that product
data['Item_Visibility'].fillna(data.groupby(by ='Item_Identifier')['Item_Visibility'].transform('mean'),inplace = True)

In [21]:
#Checking the number of missing values
data.Item_Visibility.isnull().sum()

0

In [ ]:
'Item_Type' column:

In [22]:
# checking the count of values
data.Item_Type.value_counts()

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

In [23]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])

#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drinks'})

# checking the count of values
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [24]:
#creating a separate category for non-consumable products in fat content column

data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [ ]:
'Outlet_Size' column:

In [25]:
# checking the count of values
data.Outlet_Size.value_counts()

Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [ ]:
'Outlet_Location_Type' column:

In [26]:
# checking the count of values
data.Outlet_Location_Type.value_counts()

Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

In [ ]:
'Outlet_Type' column:

In [27]:
# checking the count of values
data.Outlet_Type.value_counts()

Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64

In [ ]:
'Outlet_Establishment_Year' column:

In [28]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [ ]:
Data Visualisation

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
 13  Item_Type_Combined         14204 non-null  obj

In [ ]:
 Categorical Encoding: creation of dummy variables

In [33]:
#First creating a list of categorical columns

categorical_cols = []
for col in data.columns:
    if data[col].dtype == 'object':
        categorical_cols.append(col)
        
print(categorical_cols)

['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'source', 'Item_Type_Combined']


In [34]:
#creating list of categorical variables whose dummies are required

cat_cols = ['Item_Fat_Content', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Type_Combined']

In [35]:
#Creating Dummies for the categorical variables

for col in cat_cols:
    dummies = pd.get_dummies(data[col], prefix=col, prefix_sep='_', drop_first=True, dtype=int)
    data = pd.concat([data, dummies], axis=1)
    data.drop(columns=[col], inplace=True)
    
data.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,source,Outlet_Years,Item_Fat_Content_Non-Edible,...,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Combined_Food,Item_Type_Combined_Non-Consumable
0,FDA15,9.30,0.016047,Dairy,249.8092,1999,3735.1380,train,14,0,...,1,1,0,0,0,1,0,0,1,0
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,2009,443.4228,train,4,0,...,0,1,0,0,1,0,1,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,1999,2097.2700,train,14,0,...,1,1,0,0,0,1,0,0,1,0
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,1998,732.3800,train,15,0,...,0,0,0,0,1,0,0,0,1,0
4,NCD19,8.93,0.000000,Household,53.8614,1987,994.7052,train,26,1,...,0,0,0,0,1,1,0,0,0,1


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Item_Identifier                    14204 non-null  object 
 1   Item_Weight                        14204 non-null  float64
 2   Item_Visibility                    14204 non-null  float64
 3   Item_Type                          14204 non-null  object 
 4   Item_MRP                           14204 non-null  float64
 5   Outlet_Establishment_Year          14204 non-null  int64  
 6   Item_Outlet_Sales                  8523 non-null   float64
 7   source                             14204 non-null  object 
 8   Outlet_Years                       14204 non-null  int64  
 9   Item_Fat_Content_Non-Edible        14204 non-null  int32  
 10  Item_Fat_Content_Regular           14204 non-null  int32  
 11  Outlet_Identifier_OUT013           14204 non-null  int

In [ ]:
We have prepared the data for modelling. Before Modelling we need to export the data back into train and test data sets. I

In [ ]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

In [38]:
test.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,...,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Combined_Food,Item_Type_Combined_Non-Consumable
8523,FDW58,20.750,0.007565,107.8622,NaN,14,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0
8524,FDW14,8.300,0.038428,87.3198,NaN,6,0,1,0,1,...,0,0,0,1,0,1,0,0,1,0
8525,NCN55,14.600,0.099575,241.7538,NaN,15,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
8526,FDQ58,7.315,0.015388,155.0340,NaN,6,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
8527,FDY38,13.600,0.118599,234.2300,NaN,28,0,1,0,0,...,0,1,0,0,1,0,0,1,1,0


In [39]:
train.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_Non-Edible,Item_Fat_Content_Regular,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,...,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Type_Combined_Food,Item_Type_Combined_Non-Consumable
0,FDA15,9.30,0.016047,249.8092,3735.1380,14,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0
1,DRC01,5.92,0.019278,48.2692,443.4228,4,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
2,FDN15,17.50,0.016760,141.6180,2097.2700,14,0,0,0,0,...,1,1,0,0,0,1,0,0,1,0
3,FDX07,19.20,0.000000,182.0950,732.3800,15,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,NCD19,8.93,0.000000,53.8614,994.7052,26,1,0,1,0,...,0,0,0,0,1,1,0,0,0,1


In [ ]:
We can see that our target variable 'sales' has high correlation with 'MRP'.
Now the number of variables is 24, which is pretty high. So we shall use hybrid method of modelling. We will first use Automatic method to reduce the number of variables from 24 to 12 and then use Manual method to reduce further.
We will be checking VIF again and again so we shall create a function for checking.

In [ ]:
def calculateVIF(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [ ]:
Model 1: Model Building using all the variables